<a href="https://colab.research.google.com/github/rregmi1993/LLM-large-language-model/blob/main/03_langchain_embeddings_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r ./requirement.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

**Splitting and Embedding Text Using LangChain**

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

In [5]:
with open('churchill_speech.txt') as f:
  churchill_speech = f.read()

In [6]:
churchill_speech

'Winston Churchill 6.2.4\ng\nI would say to the House, as I said to\nthose who have joined this government:\nI have nothing to offer but blood, toil,\ntears and sweat.\n- Speech in House of Commons on May 13, 1940, three\ndays after becoming prime minister\nWe shall fight on the beaches. We shall\nfight on the landing grounds. We shall\nfight in the fields, and in the streets,\nwe shall fight in the hills. We shall never\nsurrender!\n- Speech about Dunkirk given in House of Commons on June 4, 1940\nNever in the field of human conflict was so much owed by so many to\nso few.\n- Speech made in the House of Commons on August 20, 1940, during the Battle of Britain. It\nwas a time of intense German bombing raids, and Britain had become "a whole nation fighting\nand suffering together." In this speech Churchill is expressing special gratitude toward the\nmembers of the Air Force, who were very important in turning the tide of the war.\nNever give in - never, never, never, never, in nothing g

In [9]:
chunks = text_splitter.create_documents([churchill_speech])
print(chunks[0])

page_content='Winston Churchill 6.2.4\ng\nI would say to the House, as I said to'


In [10]:
len(chunks)

34

### Embedding Cost

In [12]:
def print_embedding_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f"total tokens: {total_tokens}")
  print(f"Embedding cost in USD: {total_tokens / 1000 * 0.004:.6f}")

In [13]:
print_embedding_cost(chunks)

total tokens: 596
Embedding cost in USD: 0.002384


In [16]:
from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [17]:
vector = embedding.embed_query(chunks[0].page_content)
print(vector)

[-0.015503406908700553, -0.010032001199976472, -0.0077517034543502766, -0.01636834669426843, 0.0049078828939616015, 0.01746918118060412, -0.005022553385785553, -0.02323545200565888, 0.0024097115398328097, -0.00691952526946249, 0.0156737737514952, 0.009461926530739273, 0.013085504386621098, -0.013970102529580922, -0.008053122287800868, 0.0010164691154251095, 0.014271521363031515, 0.00033745790387109336, 0.007109550469649088, 0.001267924354155058, -0.006795026530265162, -0.0017937689998289404, 0.01843896367628387, -0.00018162935607229306, -0.011250781174050876, -0.009953370564376464, 0.023209240862469612, -0.02009021354314221, 0.0077517034543502766, -0.011008335550130938, -0.00040564570649474443, -0.01043170902641902, -0.023392713276858892, -0.003856193835444487, -0.007266812206510403, -0.011683251998157411, -0.008439724542648784, -0.0009804299576931146, 0.013904576534252951, 0.004678542841636298, 0.015398565129911277, -0.0012392568476143951, 0.008380751333118129, -0.008020359290136883, 